In [3]:
from aws_setup import *

In [ ]:
def create_instance(name, instance_type='t2.nano'):
    ami = region2ami[session.region_name]
    vpc_id, sg_id, subnet_id = get_vpc_ids(name)
    network_interfaces=[{
    'DeviceIndex': 0,
    'SubnetId': subnet_id,
    'Groups': [sg_id],
    'AssociatePublicIpAddress': True            
    }]
    block_device_mappings = [{ 
        'DeviceName': '/dev/sda1', 
        'Ebs': { 
            'VolumeSize': 128, 
            'VolumeType': 'gp2' 
        } 
    }]
    instance = ec2.create_instances(ImageId=ami, InstanceType=instance_type, 
                     MinCount=1, MaxCount=1,
                     KeyName=f'aws-key-{name}',
                     BlockDeviceMappings=block_device_mappings,
                     NetworkInterfaces=network_interfaces
                    )[0]
    instance.create_tags(Tags=[{'Key':'Name','Value':f'{name}-gpu-machine'}])
    
    print('Instance created...')
    instance.wait_until_running()
    print('Creating public IP address...')
    ec2c = session.client('ec2')
    alloc_addr = ec2c.allocate_address(Domain='vpc')
    ec2c.associate_address(InstanceId=instance.id, AllocationId=alloc_addr['AllocationId'])
    
    print('Rebooting...')
    instance.reboot()
    instance.wait_until_running()
    return instance

In [9]:
name='fast-ai'

In [24]:
def allocate_vpc_addr(instance_id):
    ec2c = session.client('ec2')
#     alloc_addr = ec2c.allocate_address(Domain='vpc')
    alloc_addr = ec2c.describe_addresses()['Addresses'][0]
    addr_id = alloc_addr['AllocationId']
#     ec2c.associate_address(InstanceId=instance_id, AllocationId=addr_id)
    return alloc_addr

In [25]:

addr = allocate_vpc_addr('sdf')

In [4]:
ec2c = session.client('ec2')
# alloc_addr = ec2c.allocate_address(Domain='vpc')
# ec2c.associate_address(InstanceId=instance.id, AllocationId=alloc_addr['AllocationId'])

In [15]:
ec2c.describe_addresses()['Addresses'][0]

{'AllocationId': 'eipalloc-164bec2a',
 'Domain': 'vpc',
 'PublicIp': '52.43.199.214'}

In [6]:
alloc_id = ec2c.describe_addresses()['Addresses'][0]['AllocationId']

In [10]:
vpc_id, sg_id, subnet_id = get_vpc_ids(name)

In [11]:
network_interface = ec2.create_network_interface(SubnetId=subnet_id, Groups=[sg_id])

In [ ]:
addr = allocate_vpc_addr(network_interface.id)

In [27]:
public_ip = addr['PublicIp']

In [ ]:
ec2c.request_spot_instances(SpotPrice)

### Adapting code shell

In [ ]:

export networkInterfaceId=`aws ec2 create-network-interface --subnet-id $subnetId --groups $securityGroupId --query 'NetworkInterface.NetworkInterfaceId' --output text`
export allocAddr=`aws ec2 allocate-address --domain vpc --query 'AllocationId' --output text`
export assocId=`aws ec2 associate-address --network-interface-id $networkInterfaceId --allocation-id $allocAddr --query 'AssociationId' --output text`
export instancePublicIp=`aws ec2 describe-addresses --query 'Addresses[?AssociationId==\`'$assocId'\`][PublicIp]' --output text`

export spotInstanceRequestId=`aws ec2 request-spot-instances --spot-price "$2" --launch-specification '{"ImageId": "'$ami'", "InstanceType": "'$1'", "KeyName": "'aws-key-$name'", "NetworkInterfaces": [{"DeviceIndex": 0, "NetworkInterfaceId": "'$networkInterfaceId'"}], "BlockDeviceMappings": [{"DeviceName": "/dev/sda1", "Ebs": {"VolumeSize": 20, "VolumeType": "gp2", "DeleteOnTermination": true}}]}' --query 'SpotInstanceRequests[0].[SpotInstanceRequestId]' --output text`
